In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [4]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]


def main():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=start_date,
            maxResults=end_date,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    """
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])
    """

    for event in events:
        start = event["start"].get("dateTime", None)
        end = event["end"].get("dateTime", None)
        summary = event["summary"]
        color_id = event.get("colorId", None)
    
        if start:
            start_datetime = datetime.datetime.fromisoformat(start)
            start_str = start_datetime.strftime("%Y-%m-%d %H:%M:%S")
        else:
            start_str = "No specific start time"
    
        if end:
            end_datetime = datetime.datetime.fromisoformat(end)
            end_str = end_datetime.strftime("%Y-%m-%d %H:%M:%S")
            duration = end_datetime - start_datetime
            duration_str = str(duration)
        else:
            end_str = "No specific end time"
            duration_str = "No specific duration"
    
        print(f"Start: {start_str}")
        print(f"End: {end_str}")
        print(f"Summary: {summary}")
        print(f"Duration: {duration_str}")
        print(f"Color ID: {color_id}")
        print()

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Getting the events between specified dates


ValueError: invalid literal for int() with base 10: '2024-03-16T00:00:00Z'